# Applied Data Science Capstone Project
##### Segmenting and Clustering Neighborhoods in Toronto
- Part 1 - Explore and cluster the neighborhoods in Toronto
- Part 2 - Geospacial data for the neighborhoods in Toronto
- Part 3 - Clustering and exploring

## Part 1 - Explore and cluster the neighborhoods in Toronto

In [3]:
# Import libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium=0.5.0 --yes
!pip install folium
import folium # map rendering library

import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


### Option 1
Use BeautifulSoup to parse HTML into DataFrame

In [ ]:
"""
# Import urllib to get html file
import urllib as ub 

# Install and import beautifulsoup to manage web page content
!pip install beautifulsoup4
from bs4 import BeautifulSoup

# Install lxml parser
!pip install lxml
"""

In [ ]:
"""
# Fetch the html file
response = ub.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
html_doc = response.read()

# Parse the html file
soup = BeautifulSoup(html_doc, 'lxml')

# extract table data into dataframe
table = soup.find('table', class_='wikitable sortable')
output_rows = []
for table_row in table.findAll('tr'):
    columns = table_row.findAll('td')
    output_row = []
    for column in columns:
        c = column.text
        c = c.replace('\n', '')
        output_row.append(c)
    output_rows.append(output_row)
df = pd.DataFrame(output_rows[1:]) # fist row [0] is dummy header, so it is omitted
df.columns = ['Postal code','Borough','Neighborhood'] # assign column names
print(df.shape)
df.head()
"""

### Option 2
Use direct import of HTML table into DataFrame with pandas
This is easy and simply way to read HTML table into DataFrame when we know/inspect the structure of the HTML file.

In [4]:
# Direct import of HTML table data into DataFrame
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)
df = df[0]
print(df.shape)
df.head()

(180, 3)


,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### DataFrame Wrangling
- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. These rows will be combined into one row with the neighborhoods separated with a comma.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [5]:
df.rename(columns={'Postal code': 'PostalCode'}, inplace=True)
indexNames = df[df['Borough'] == 'Not assigned'].index
df.drop(indexNames, inplace=True)
df.sort_values(by=['PostalCode'], inplace=True)
df.reset_index(drop=True, inplace=True)
print(df.shape)
df.head()

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
df.nunique()

PostalCode      103
Borough          10
Neighborhood     98
dtype: int64

In [7]:
df['Neighborhood'] = pd.Series(df['Neighborhood']).str.replace(' / ', ', ')
print(df.shape)
df.head()

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Finalize DataFrame Wrangling
- Last cell of this section of the notebook, dicply .shape method to print the number of rows of the dataframe.

In [8]:
df.shape

(103, 3)

## Part 2 - Geospacial data for the neighborhoods in Toronto

### Option 1
Using geocoder for Geo Data (this option will not be used, as service is not reliable)

In [ ]:
"""
# Install and import geocoder
!pip install geocoder
import geocoder #
"""

In [ ]:
"""
df['Latitude'] = df2['PostalCode']
df['Longitude'] = df2['PostalCode']

# Append DataFrame with Latitude and Longitude
for index, row in df.iterrows():
    # access data using column names
    postal_code = row['PostalCode']

    # initialize variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    row['latitude'] = lat_lng_coords[0]
    row['longitude'] = lat_lng_coords[1]

print(df.shape)
df.head()
"""

### Option 2
Load CSV file for Geo Data

In [9]:
df_geo = pd.read_csv('https://cocl.us/Geospatial_data', header=0)
# df_geo = df_geo[0]
print(df_geo.shape)
df_geo.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Prepare Geo Data
DataFrame with Geo Dada is prepared and appended to the mail DataFrame  
By inspecting the data, Geo Data will be only appended (no need to loop and match)

In [10]:
# Prepare Geo DataFrame
df_geo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_geo.sort_values(by=['PostalCode'], inplace=True)
df_geo.reset_index(drop=True, inplace=True)
print(df_geo.shape)
df_geo.head()

(103, 3)


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
df_neigh = df
#df_neigh['Latitude'] = df_geo['Latitude']
#df_neigh['Longitude'] = df_geo['Longitude']
df_neigh = df_neigh.join(df_geo.set_index('PostalCode'), on='PostalCode')
print(df_neigh.shape)
df_neigh.head()

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Part 3 - Clustering and exploring

Use geopy library to get the latitude and longitude values of Toronto.
In order to define an instance of the geocoder, define a user_agent as toronto_explorer.

In [12]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In order to simplfy analysis, let's use only Central Toronto and Downtown Toronto

In [13]:
df_neigh_central = df_neigh[df_neigh['Borough'].isin(['Central Toronto', 'Downtown Toronto'])].reset_index(drop=True)
print(df_neigh_central.shape)
df_neigh_central.head()

(28, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160


Map of Toronto with neighborhoods superimposed on top

In [14]:
# create map
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_neigh_central['Latitude'], df_neigh_central['Longitude'], df_neigh_central['Borough'], df_neigh_central['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

Foursquare Credentials and Version

In [15]:
# The code was removed by Watson Studio for sharing.

Function to get details about nearby venues

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius, LIMIT):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood_Latitude', 
                  'Neighborhood_Longitude', 
                  'Venue', 
                  'Venue_Latitude', 
                  'Venue_Longitude', 
                  'Venue_Category']
    
    return(nearby_venues)

Run function to get details about venues for Central Toronto and DOwntown Toronto

In [17]:
# veunue radios is 500m, number of requests is limited to 100
#radius=500
#LIMIT=100

# create new dataframe by getting nerby venues
df_venues = getNearbyVenues(names=df_neigh_central['Neighborhood'], latitudes=df_neigh_central['Latitude'], longitudes=df_neigh_central['Longitude'], radius=500, LIMIT=100)
print(df_venues.shape)
df_venues.head()

(1349, 7)


,Neighborhood,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Davisville North,43.712751,-79.390197,Homeway Restaurant & Brunch,43.712641,-79.391557,Breakfast Spot
4,Davisville North,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park


Let's check how many venues were returned for each neighborhood

In [18]:
df_venues.groupby(['Neighborhood']).count()

,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",18,18,18,18,18,18
Central Bay Street,64,64,64,64,64,64
Christie,18,18,18,18,18,18
Church and Wellesley,76,76,76,76,76,76
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,37,37,37,37,37,37
Davisville North,8,8,8,8,8,8
"First Canadian Place, Underground city",100,100,100,100,100,100


Let's find out how many unique categories can be curated from all the returned venues

In [19]:
print('There are {} uniques categories.'.format(len(df_venues['Venue_Category'].unique())))

There are 215 uniques categories.


Let's do One Hot Encoding for each Venue Category
- Pls note in the venues there is a category 'Neighborhood', so one hot encoding will create column with name 'Neighborhood'
- This column will be renamed to 'Neighborhood_Venue'

In [25]:
# one hot encoding
df_onehot = pd.get_dummies(df_venues['Venue_Category'], prefix="", prefix_sep="")

# since there is a venue category, rename it
df_onehot.rename(columns={'Neighborhood': 'Neighborhood Venue'}, inplace=True)

# create dataframe with neighborhood column, and columns for each venue category
df_onehot = pd.concat([df_venues['Neighborhood'], df_onehot], axis=1)
print(df_onehot.shape)
df_onehot.head()

(1349, 216)


,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Cable Car,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,

Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [26]:
df_grouped = df_onehot.groupby('Neighborhood').mean().reset_index()
print(df_grouped.shape)
df_grouped.head()

(28, 216)


,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Cable Car,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.017241,0.000000,0.000000,0.0,0.000000,0.017241,0.000000,0.017241,0.034483,0.0,0.000000,0.0,0.017241,0.017241,0.0,0.034483,0.0,0.0,0.017241,0.000000,0.000000,0.000000,0.0,0.017241,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.034483,0.000000,0.000000,0.034483,0.0,0.0,0.0,0.017241,0.051724,0.068966,0.0,0.0,0.0,0.0,0.0,0.0,0.017241,0.000000,0.017241,0.0,0.0,0.0,0.017241,0.0,0.000000,0.0,0.000000,0.0,0.017241,0.000000,0.000000,0.000000,0.0,0.000000,0.017241,0.0,0.0,0.0,0.000000,0.034483,0.000000,0.0,0.017241,0.0,0.000000,0.0,0.0,0.017241,0.017241,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.017241,0.017241,0.017241,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.017241,0.0,0.0,0.00000,0.000000,0.0,0.0,0.034483,0.017241,0.017241,0.0,0.017241,0.000000,0.0,0.0,0.0,0.0,0.017241,0.

Let's write a function to sort the venues in descending order.

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Let's create the new dataframe and display the top 10 venues for each neighborhood.

In [55]:
# top 10 venues, 
top = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']

# create columns according to number of top venues
for i in np.arange(top):
    try:
        columns.append('{}{} Most Common Venue'.format(i+1, indicators[i]))
    except:
        columns.append('{}th Most Common Venue'.format(i+1))

# create a new dataframe
df_top_venues = pd.DataFrame(columns=columns)
df_top_venues['Neighborhood'] = df_grouped['Neighborhood']

for ind in np.arange(df_grouped.shape[0]):
    df_top_venues.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], top)

print(df_top_venues.shape)
df_top_venues.head()

(28, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Italian Restaurant,Cheese Shop,Restaurant,Seafood Restaurant,Beer Bar,Café
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Airport,Boutique,Bar,Rental Car Location,Plane,Boat or Ferry,Coffee Shop
2,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Restaurant,Bar,Bubble Tea Shop,Burger Joint,Fried Chicken Joint
3,Christie,Grocery Store,Café,Park,Restaurant,Gas Station,Diner,Italian Restaurant,Baby Store,Athletics & Sports,Candy Store
4,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Yoga Studio,Men's Store,Hotel,Pub,Gay Bar,Gastropub


Run k-means to cluster the neighborhood into 5 clusters.

In [56]:
# set number of clusters
k = 3

# drop neighborhood column
df_grouped_clusters = df_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(df_grouped_clusters)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 2, 1,
       1, 1, 1, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [57]:
# add clusters labels into top venues
df_top_venues.insert(0, 'Cluster Labels', kmeans.labels_)

# merge central toronto data (incl. latitude/longitude for each neighborhood) with clusters data
df_neigh_central_top_venues = df_neigh_central
df_neigh_central_top_venues = df_neigh_central_top_venues.join(df_top_venues.set_index('Neighborhood'), on='Neighborhood')
print(df_neigh_central_top_venues.shape)
df_neigh_central_top_venues.head()

(28, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Swim School,Bus Line,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Gym,Hotel,Convenience Store,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Park,Gay Bar,Gastropub
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,1,Clothing Store,Coffee Shop,Yoga Studio,Mexican Restaurant,Seafood Restaurant,Salon / Barbershop,Restaurant,Rental Car Location,Park,Fast Food Restaurant
3,M4S,Central Toronto,Davisville,43.704324,-79.388790,1,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Café,Gym,Toy / Game Store,Sushi Restaurant,Italian Restaurant,Thai Restaurant
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,0,Park,Playground,Tennis Court,Restaurant,Creperie,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


Finally, let's visualize the resulting clusters

In [58]:
# create map
toronto_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_neigh_central_top_venues['Latitude'], df_neigh_central_top_venues['Longitude'], df_neigh_central_top_venues['Neighborhood'], df_neigh_central_top_venues['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(toronto_map_clusters)
       
toronto_map_clusters

#### Let's examine each cluster and determine the discriminating venue categories that distinguish each cluster.

Cluster 0

In [59]:
df_neigh_central_top_venues.loc[df_neigh_central_top_venues['Cluster Labels'] == 0, 
                                df_neigh_central_top_venues.columns[[2] + list(range(5, df_neigh_central_top_venues.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Lawrence Park,0,Park,Swim School,Bus Line,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
4,"Moore Park, Summerhill East",0,Park,Playground,Tennis Court,Restaurant,Creperie,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
6,Rosedale,0,Park,Trail,Playground,Creperie,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
19,Forest Hill North & West,0,Park,Jewelry Store,Trail,Sushi Restaurant,Bus Line,Dance Studio,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


Cluster 1

In [60]:
df_neigh_central_top_venues.loc[df_neigh_central_top_venues['Cluster Labels'] == 1, 
                                df_neigh_central_top_venues.columns[[2] + list(range(5, df_neigh_central_top_venues.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Davisville North,1,Gym,Hotel,Convenience Store,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Park,Gay Bar,Gastropub
2,North Toronto West,1,Clothing Store,Coffee Shop,Yoga Studio,Mexican Restaurant,Seafood Restaurant,Salon / Barbershop,Restaurant,Rental Car Location,Park,Fast Food Restaurant
3,Davisville,1,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Café,Gym,Toy / Game Store,Sushi Restaurant,Italian Restaurant,Thai Restaurant
5,"Summerhill West, Rathnelly, South Hill, Forest...",1,Coffee Shop,Pub,Light Rail Station,Pizza Place,Supermarket,Sushi Restaurant,Sports Bar,Bank,Fried Chicken Joint,Bagel Shop
7,"St. James Town, Cabbagetown",1,Coffee Shop,Restaurant,Pub,Italian Restaurant,Café,Bakery,Pizza Place,Pharmacy,Chinese Restaurant,Caribbean Restaurant
8,Church and Wellesley,1,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Yoga Studio,Men's Store,Hotel,Pub,Gay Bar,Gastropub
9,"Regent Park, Harbourfront",1,Coffee Shop,Pub,Bakery,Park,Restaurant,Café,Theater,Breakfast Spot,Farmers Market,Mexican Restaurant
10,"Garden District, Ryerson",1,Coffee Shop,Clothing Store,Café,Restaurant,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Middle Eastern Restaurant,Tea Room,Bookstore
11,St. James Town,1,Coffee Shop,Café,Italian Restaurant,Gastropub,Cocktail Bar,American Restaurant,Farmers Market,Department Store,Clothing Store,Hotel
12,Berczy Park,1,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Italian Restaurant,Cheese Shop,Restaurant,Seafood Restaurant,Beer Bar,Café


Cluster 2

In [61]:
df_neigh_central_top_venues.loc[df_neigh_central_top_venues['Cluster Labels'] == 2, 
                                df_neigh_central_top_venues.columns[[2] + list(range(5, df_neigh_central_top_venues.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Roselawn,2,Garden,Yoga Studio,Dance Studio,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


#### Colclusion
- Cluster 0 is Sport and Recreation Area
- Cluster 1 is Business Area
- Cluster 2 is Garden and Walking Area